In [ ]:
from Solve import psi
import numpy as np
n = 9999
wavefunction, x, t = psi([-200, 4096, -10, n])
print(wavefunction[50,50])

import matplotlib.pyplot as plt
plt.plot(x, abs(wavefunction[n//2]))
plt.xlim(-40, 40)
#plt.ylim(0, 5)
plt.show()

In [4]:
from Res_visualisation import *
from Parameters import pars_YanPengPhysRevA_78_033821
from Hydrogen import Hydrogen
from Field import Field

params = pars_YanPengPhysRevA_78_033821()[0]
H = Hydrogen()
F = Field.Pulse(params)
field = F(t, 'Real')
pot = H.potential(x)
plot_HH_spectrum(x, t, wavefunction, params, pot, field, Z=1)
imshow_time_frequency_characteristics(x, t, wavefunction, params, pot, field)

c:\Users\user\Programming\Git\HHG\Res_visualisation.py:8: RuntimeWarning: overflow encountered in multiply
  a = np.trapz(-wavefunction * dV_dx[np.newaxis,:] * np.conj(wavefunction), x[np.newaxis,:], axis=1)
c:\Users\user\Programming\Git\HHG\Res_visualisation.py:8: RuntimeWarning: invalid value encountered in multiply
  a = np.trapz(-wavefunction * dV_dx[np.newaxis,:] * np.conj(wavefunction), x[np.newaxis,:], axis=1)
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:4979: RuntimeWarning: overflow encountered in add
  ret = (d * (y[tuple(slice1)] + y[tuple(slice2)]) / 2.0).sum(axis)
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:4979: RuntimeWarning: invalid value encountered in add
  ret = (d * (y[tuple(slice1)] + y[tuple(slice2)]) / 2.0).sum(axis)
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:4979: RuntimeWarning: invalid value encou

MemoryError: Unable to allocate 106. GiB for an array with shape (12001, 12001, 99) and data type float64